In [21]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import layers

In [22]:
import matplotlib.pyplot as plt
import numpy as np

# regression

In [23]:
california_housing=fetch_california_housing()

گرفتن خود داده

In [24]:
X_train0, X_test, y_train0, y_test = train_test_split(
                 california_housing["data"],
                 california_housing["target"])

حال نیاز است که یک
validation
از همون داده خودمون بگیریم

In [25]:
X_train1, X_validaion, y_train1, y_validaion = train_test_split(
                                                                 X_train0,
                                                                 y_train0)

In [26]:
X_test.shape

(5160, 8)

In [27]:
X_validaion.shape

(3870, 8)

In [28]:
X_train0.shape

(15480, 8)

In [29]:
X_train1.shape

(11610, 8)

In [30]:
sc=StandardScaler()
X_train_s = sc.fit_transform(X_train1)
#ما چون از داده جدید مثلا هیچی نمیدونیم و توی غالب داده های قبلی باید ببریم مثلا پس دیگ (فیت) نمی کنیم در واقع فقط بر اساس داده هایی که قبلا توی ترین داشتیم ضرب میکنه که فقط ترنسفورم روی نورمالایز یا استاندارد سازی که قبلا داشتیم می کنیم
X_validaion_s= sc.transform(X_validaion)
X_test_s= sc.transform(X_test)

In [31]:
model = keras.models.Sequential([
    layers.Dense(75, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(50, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(30, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    
    layers.Dense(1, activation='linear')  # خروجی برای رگرسیون
])

In [32]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

In [20]:
history=model.fit(
    x=X_train_s,
    y=y_train1,
    epochs=30,
    validation_data=(X_validaion_s,y_validaion)
)

Epoch 1/30
363/363 [==============================] - 3s 5ms/step - loss: 2.2364 - mae: 1.1367 - val_loss: 0.6114 - val_mae: 0.5866
Epoch 2/30
363/363 [==============================] - 2s 4ms/step - loss: 0.7714 - mae: 0.6662 - val_loss: 0.5330 - val_mae: 0.5415
Epoch 3/30
363/363 [==============================] - 2s 5ms/step - loss: 0.6467 - mae: 0.6023 - val_loss: 0.4907 - val_mae: 0.5261
Epoch 4/30
363/363 [==============================] - 2s 5ms/step - loss: 0.5932 - mae: 0.5740 - val_loss: 0.4691 - val_mae: 0.5058
Epoch 5/30
363/363 [==============================] - 2s 5ms/step - loss: 0.5570 - mae: 0.5552 - val_loss: 0.4552 - val_mae: 0.4886
Epoch 6/30
363/363 [==============================] - 2s 5ms/step - loss: 0.5387 - mae: 0.5436 - val_loss: 0.4268 - val_mae: 0.4772
Epoch 7/30
363/363 [==============================] - 3s 8ms/step - loss: 0.5109 - mae: 0.5285 - val_loss: 0.4126 - val_mae: 0.4686
Epoch 8/30
363/363 [==============================] - 2s 7ms/step - loss: 0.

## functional api

In [33]:
# از یک به بعد همه رو حساب کنه که تاپلی باشه
X_train1.shape[1:]

(8,)

فرم کلی اینکه همی داده هارو بدیم

In [34]:
# Just for demonstration purposes.
input_ = keras.layers.Input(shape=X_train1.shape[1:])
hidden_layer1=keras.layers.Dense(50, activation="relu")(input_)
hidden_layer2=keras.layers.Dense(10, activation="relu")(hidden_layer1)
concatenate_layer=keras.layers.Concatenate()([input_, hidden_layer2])
output_=keras.layers.Dense(1)(concatenate_layer)


model_conc = keras.Model(inputs=[input_],
                    outputs=[output_],
                    name="history_concat")

مشخص میکنیم چه داده هایی باید برن توی اینپوت 1 و چقدر برن توی اینپوت 2 

In [35]:
X_train_s_1 , X_train_s_2 = X_train_s [:,:6] , X_train_s [:,-4:]
X_validaion_s_1 , X_validaion_s_2 = X_validaion_s [:,:6] , X_validaion_s [:,-4:]

X_test_s_1 , X_test_s_2 = X_test_s [:,:6] , X_test_s [:,-4:]

فرمتی که می خوایم مثلا 6 تا ستون از یک داده بره توی دیپ و 4 تاش (که اور لپ هم دارن باهم دیگه) بره تو لایه کان کتینت و در لایه آخر مرج شن

In [36]:
# Just for demonstration purposes.
input_1 = keras.layers.Input(shape=[6])
input_2 = keras.layers.Input(shape=[4])


hidden_layer1=keras.layers.Dense(50, activation="relu")(input_1)
hidden_layer2=keras.layers.Dense(10, activation="relu")(hidden_layer1)

concatenate_layer=keras.layers.Concatenate()([input_2, hidden_layer2])

output_=keras.layers.Dense(1, name='output')(concatenate_layer)

helper_output_=keras.layers.Dense(1, name='helper_output')(hidden_layer2)

 # به نحوه ترتیب اینپوت ها نگاه کن که کدومو میبری توی شبکه و کدوم لایه را اسکیپ میکنه تا لایه کاکنت
model_conc = keras.Model(inputs=[input_1,input_2],
                    outputs=[output_,helper_output_],
                    name="history_concat")

In [37]:
model_conc.compile(
    optimizer="sgd",
    loss=['mean_squared_error','MAE'],
    # مشحص کنیم هر کدام از خروجی ها برای ما چه وزنی یا ارزشی دارند
    loss_weights=[0.8,0.2] ,
    metrics=['mean_absolute_error'],
)

In [38]:
history_conc=model_conc.fit(
    x=(X_train_s_1 , X_train_s_2 ),
    #چون ما دوتا خروجی داریم پس دوتا وای هم داریم 
    y=(y_train1,y_train1),
    epochs=30,
    validation_data=  ((X_validaion_s_1 , X_validaion_s_2) ,
                      (y_validaion,y_validaion))
    )

Epoch 1/30
363/363 [==============================] - 3s 7ms/step - loss: 0.9650 - output_loss: 0.9964 - helper_output_loss: 0.8394 - output_mean_absolute_error: 0.6801 - helper_output_mean_absolute_error: 0.8394 - val_loss: 2.1403 - val_output_loss: 2.4996 - val_helper_output_loss: 0.7033 - val_output_mean_absolute_error: 0.5750 - val_helper_output_mean_absolute_error: 0.7033
Epoch 2/30
363/363 [==============================] - 2s 7ms/step - loss: 8.5372 - output_loss: 10.5259 - helper_output_loss: 0.5822 - output_mean_absolute_error: 0.6694 - helper_output_mean_absolute_error: 0.5822 - val_loss: 1.5549 - val_output_loss: 1.8004 - val_helper_output_loss: 0.5726 - val_output_mean_absolute_error: 0.5620 - val_helper_output_mean_absolute_error: 0.5726
Epoch 3/30
363/363 [==============================] - 2s 6ms/step - loss: 0.7804 - output_loss: 0.8375 - helper_output_loss: 0.5518 - output_mean_absolute_error: 0.5343 - helper_output_mean_absolute_error: 0.5518 - val_loss: 1.2015 - val_o

# subclass api

In [39]:
# تعریف کلاس مدل که از keras.Model ارث بری می‌کند
class Wide_deep_end_ann(keras.Model):
    # اینجا ما میتونیم چیزای زیادی را برای اینیشیلایز کردن بهش بدیم
    # مثلا هرچی که می خواید موقع اینیشیلایز کردن مدل به عنوان ورودی بهش بدید
    # مثلا میتونید تعداد نورون هارو بدید بهش یا مثلا میتونید اکتیویژن هارو بهش بدید
    # وقتی توی پایتون دوتا ستاره و کیورد آرگیومنت را می نویسیم
    # می تواند به تعداد دلخواه کیورد آرگیومنت بگیره
    # حالا ما اینو مینویسیم که به سوپر بدیمش

    # متد سازنده‌ی کلاس که امکان تعریف پارامترهای دلخواه برای مدل را فراهم می‌کند
    def __init__(self, activation="relu", **kwargs):
        """
        ورودی‌های متد:
        activation: تابع فعال‌سازی که برای لایه‌ها استفاده می‌شود (پیش‌فرض 'relu')
        **kwargs: سایر پارامترهای اختیاری که می‌توان به کلاس پایه keras.Model پاس داد
        """
        # مقداردهی اولیه‌ی کلاس پایه با استفاده از پارامترهای داده شده
        # ما این پایینی رو مینویسیم که هرچی توی کراس مادلز هست را بیاریمش
        # به عنوان ورودی کیورد آرگیومنت را میدهیم
        super().__init__(**kwargs)
        
        # تعریف لایه‌ی Dense با 50 نورون و تابع فعال‌سازی مشخص شده
        self.hidden_layer1 = keras.layers.Dense(50, activation=activation)
        
        # تعریف لایه‌ی Dense دوم با 10 نورون و تابع فعال‌سازی مشخص شده
        self.hidden_layer2 = keras.layers.Dense(10, activation=activation)

        # تعریف لایه‌ی خروجی نهایی با یک نورون (برای پیش‌بینی اصلی)
        self.final_output = keras.layers.Dense(1)

        # تعریف لایه‌ی کمکی خروجی با یک نورون (برای پیش‌بینی کمکی)
        self.helper_output = keras.layers.Dense(1)

    # کار مدل تازه اینجا شروع میشه
    # اینجا درواقع کار گرفتن اینپوت از مخاطب رو میکنه
    # ما در بالا مدل را ساختیم با ویژگی های دلخواهمون
    # و در پایین می خواهیم تازه دیتا خودمون رو بدهیم
    # در واقع این با خط مدل_کانکت که در بالا تر نوشته بودیم یکی است

    # متدی که رفتار مدل را هنگام دریافت ورودی‌ها تعریف می‌کند
    def call(self, inputs___):
        """
        ورودی‌های متد:
        inputs: لیستی شامل دو ورودی جداگانه (input_1 و input_2)
        خروجی:
        final_out: خروجی نهایی مدل
        helper_out: خروجی کمکی مدل
        """
        # دریافت ورودی‌ها و تخصیص به دو متغیر جداگانه
        input_1, input_2 = inputs___

        # عبور اولین ورودی از لایه‌ی hidden_layer1
        h1_out = self.hidden_layer1(input_1)

        # عبور خروجی قبلی از لایه‌ی hidden_layer2
        h2_out = self.hidden_layer2(h1_out)

        # ادغام خروجی دومین لایه مخفی با ورودی دوم
        con_out = keras.layers.concatenate([h2_out, input_2])

        # تولید خروجی نهایی مدل با استفاده از لایه final_output
        final_out = self.final_output(con_out)

        # تولید خروجی کمکی مدل با استفاده از لایه helper_output
        helper_out = self.helper_output(h2_out)

        # بازگرداندن خروجی‌های مدل (خروجی نهایی و کمکی)
        return final_out, helper_out


فقط از کد پایین که همون کد سابقمون است نگاه کن که بیشتر متوجه بشی که چکار کردیم

In [40]:
# Just for demonstration purposes.
input_1 = keras.layers.Input(shape=[6])
input_2 = keras.layers.Input(shape=[4])


hidden_layer1=keras.layers.Dense(50, activation="relu")(input_1)
hidden_layer2=keras.layers.Dense(10, activation="relu")(hidden_layer1)

concatenate_layer=keras.layers.Concatenate()([input_2, hidden_layer2])

output_=keras.layers.Dense(1, name='output')(concatenate_layer)

helper_output_=keras.layers.Dense(1, name='helper_output')(hidden_layer2)

 # به نحوه ترتیب اینپوت ها نگاه کن که کدومو میبری توی شبکه و کدوم لایه را اسکیپ میکنه تا لایه کاکنت
model_conc = keras.Model(inputs=[input_1,input_2],
                    outputs=[output_,helper_output_],
                    name="history_concat")

In [41]:
model_sub=Wide_deep_end_ann()

In [42]:
model_sub.compile(
    optimizer="sgd",
    loss=['mean_squared_error','MAE'],
    # مشحص کنیم هر کدام از خروجی ها برای ما چه وزنی یا ارزشی دارند
    loss_weights=[0.8,0.2] ,
    metrics=['mean_absolute_error'],
)

In [43]:
history_sub=model_sub.fit(
    x=(X_train_s_1 , X_train_s_2 ),
    #چون ما دوتا خروجی داریم پس دوتا وای هم داریم 
    y=(y_train1,y_train1),
    epochs=30,
    validation_data=  ((X_validaion_s_1 , X_validaion_s_2) ,
                      (y_validaion,y_validaion))
)

Epoch 1/30
363/363 [==============================] - 4s 8ms/step - loss: 1.0833 - output_1_loss: 1.0958 - output_2_loss: 1.0333 - output_1_mean_absolute_error: 0.7478 - output_2_mean_absolute_error: 1.0333 - val_loss: 0.8395 - val_output_1_loss: 0.8999 - val_output_2_loss: 0.5981 - val_output_1_mean_absolute_error: 0.5756 - val_output_2_mean_absolute_error: 0.5981
Epoch 2/30
363/363 [==============================] - 5s 14ms/step - loss: 0.6345 - output_1_loss: 0.6490 - output_2_loss: 0.5767 - output_1_mean_absolute_error: 0.5524 - output_2_mean_absolute_error: 0.5767 - val_loss: 0.5454 - val_output_1_loss: 0.5439 - val_output_2_loss: 0.5517 - val_output_1_mean_absolute_error: 0.5361 - val_output_2_mean_absolute_error: 0.5517
Epoch 3/30
363/363 [==============================] - 6s 15ms/step - loss: 0.5403 - output_1_loss: 0.5388 - output_2_loss: 0.5465 - output_1_mean_absolute_error: 0.5202 - output_2_mean_absolute_error: 0.5465 - val_loss: 0.5199 - val_output_1_loss: 0.5164 - val_ou

In [ ]:
model_conc.summary()

Model: "history_concat"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 dense_8 (Dense)             (None, 50)                   350       ['input_4[0][0]']             
                                                                                                  
 input_5 (InputLayer)        [(None, 4)]                  0         []                            
                                                                                                  
 dense_9 (Dense)             (None, 10)                   510       ['dense_8[0][0]']             
                                                                                     

In [ ]:
model_sub.summary()

Model: "wide_deep_end_ann"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            multiple                  350       
                                                                 
 dense_11 (Dense)            multiple                  510       
                                                                 
 dense_12 (Dense)            multiple                  15        
                                                                 
 dense_13 (Dense)            multiple                  11        
                                                                 
Total params: 886 (3.46 KB)
Trainable params: 886 (3.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## save model

In [ ]:
# ذخیره مدل به فرمت TensorFlow SavedModel
model_sub.save('housing_reg_model_sub', save_format='tf')

INFO:tensorflow:Assets written to: housing_reg_model_sub\assets


INFO:tensorflow:Assets written to: housing_reg_model_sub\assets


In [ ]:
# ذخیره و بارگذاری وزن‌های مدل
model_conc.save_weights('housing_reg_model_sub_weights.h5')


In [ ]:
model_conc.save("housing_reg_model_f.h5")

D:\anacoda\envs\a.i\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# ایجاد دوباره مدل و بارگذاری وزن‌ها
model_sub_reg1 = Wide_deep_end_ann()
model_sub_reg1.load_weights('housing_reg_model_sub_weights.h5')


In [ ]:
model_sub_reg2=keras.models.load_model('housing_reg_model_sub')
model_conc_load=keras.models.load_model('housing_reg_model_f.h5')

In [ ]:
model_sub_reg2.summary()

Model: "wide_deep_end_ann"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            multiple                  350       
                                                                 
 dense_11 (Dense)            multiple                  510       
                                                                 
 dense_12 (Dense)            multiple                  15        
                                                                 
 dense_13 (Dense)            multiple                  11        
                                                                 
Total params: 886 (3.46 KB)
Trainable params: 886 (3.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Callback API

### 1_ModelCheckpoint

In [44]:
model_Checkpoint_callback = keras.callbacks.ModelCheckpoint('model_cb_regresion_housing.h5'
                                                            ,save_best_only=True)
                                                

In [45]:
model_conc.compile(
    optimizer="sgd",
    loss=['mean_squared_error','MAE'],
    # مشحص کنیم هر کدام از خروجی ها برای ما چه وزنی یا ارزشی دارند
    loss_weights=[0.8,0.2] ,
    metrics=['mean_absolute_error'],
)

In [46]:
history_conc=model_conc.fit(
    x=(X_train_s_1 , X_train_s_2 ),
    #چون ما دوتا خروجی داریم پس دوتا وای هم داریم 
    y=(y_train1,y_train1),
    epochs=10,
    validation_data=  ((X_validaion_s_1 , X_validaion_s_2) ,
                      (y_validaion,y_validaion)),
    callbacks=[model_Checkpoint_callback]
    )

Epoch 1/10
363/363 [==============================] - 7s 17ms/step - loss: 3.4462 - output_loss: 3.8779 - helper_output_loss: 1.7197 - output_mean_absolute_error: 0.8893 - helper_output_mean_absolute_error: 1.7197 - val_loss: 10.3419 - val_output_loss: 12.6089 - val_helper_output_loss: 1.2737 - val_output_mean_absolute_error: 0.6950 - val_helper_output_mean_absolute_error: 1.2737
Epoch 2/10
363/363 [==============================] - 6s 17ms/step - loss: 32.8350 - output_loss: 40.7479 - helper_output_loss: 1.1831 - output_mean_absolute_error: 0.6952 - helper_output_mean_absolute_error: 1.1831 - val_loss: 4.8015 - val_output_loss: 5.7298 - val_helper_output_loss: 1.0884 - val_output_mean_absolute_error: 0.6061 - val_helper_output_mean_absolute_error: 1.0884
Epoch 3/10
363/363 [==============================] - 6s 17ms/step - loss: 17.0594 - output_loss: 21.0630 - helper_output_loss: 1.0448 - output_mean_absolute_error: 0.7319 - helper_output_mean_absolute_error: 1.0448 - val_loss: 2.3281

### 2_EarlyStopping

In [47]:
earlystopping_callback_ = keras.callbacks.EarlyStopping(monitor="val_loss",
                                                        patience=5,
                                                      restore_best_weights=True)

In [48]:
history_conc=model_conc.fit(
    x=(X_train_s_1 , X_train_s_2 ),
    #چون ما دوتا خروجی داریم پس دوتا وای هم داریم 
    y=(y_train1,y_train1),
    epochs=40,
    validation_data=  ((X_validaion_s_1 , X_validaion_s_2) ,
                      (y_validaion,y_validaion)),
    callbacks=[model_Checkpoint_callback,earlystopping_callback_]
    )

Epoch 1/40
363/363 [==============================] - 6s 16ms/step - loss: 2488.5481 - output_loss: 3110.4592 - helper_output_loss: 0.8901 - output_mean_absolute_error: 10.2149 - helper_output_mean_absolute_error: 0.8901 - val_loss: 4167.2783 - val_output_loss: 5208.8774 - val_helper_output_loss: 0.8719 - val_output_mean_absolute_error: 8.2624 - val_helper_output_mean_absolute_error: 0.8719
Epoch 2/40
363/363 [==============================] - 6s 16ms/step - loss: 1068.2433 - output_loss: 1335.0841 - helper_output_loss: 0.8833 - output_mean_absolute_error: 5.8417 - helper_output_mean_absolute_error: 0.8833 - val_loss: 1912.3839 - val_output_loss: 2390.2639 - val_helper_output_loss: 0.8698 - val_output_mean_absolute_error: 5.3006 - val_helper_output_mean_absolute_error: 0.8698
Epoch 3/40
363/363 [==============================] - 6s 16ms/step - loss: 502.7529 - output_loss: 628.2210 - helper_output_loss: 0.8811 - output_mean_absolute_error: 3.8415 - helper_output_mean_absolute_error: 0.

### 3_ mycallback

In [49]:
class MyCallback(keras.callbacks.Callback):
    '''مراجعه به سایت پایین برای تعیین اور رایت متدها
    https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback
    '''
    def on_train_end(self, logs=None):
        print('اند تمام شد مثلا')
    def on_epoch_end (self,epoch,logs):
        print(logs['val_loss'],'______$$____')

In [50]:
MyCallback_mehr=MyCallback()

In [51]:
history_conc=model_conc.fit(
    x=(X_train_s_1 , X_train_s_2 ),
    #چون ما دوتا خروجی داریم پس دوتا وای هم داریم 
    y=(y_train1,y_train1),
    epochs=40,
    validation_data=  ((X_validaion_s_1 , X_validaion_s_2) ,
                      (y_validaion,y_validaion)),
    callbacks=[model_Checkpoint_callback,earlystopping_callback_,MyCallback_mehr]
    )

Epoch 1/40
363/363 [==============================] - 6s 16ms/step - loss: 0.9751 - output_loss: 0.9988 - helper_output_loss: 0.8802 - output_mean_absolute_error: 0.7678 - helper_output_mean_absolute_error: 0.8802 - val_loss: 0.9512 - val_output_loss: 0.9716 - val_helper_output_loss: 0.8699 - val_output_mean_absolute_error: 0.7613 - val_helper_output_mean_absolute_error: 0.8699
Epoch 2/40
363/363 [==============================] - 6s 16ms/step - loss: 0.9761 - output_loss: 1.0001 - helper_output_loss: 0.8802 - output_mean_absolute_error: 0.7680 - helper_output_mean_absolute_error: 0.8802 - val_loss: 0.9571 - val_output_loss: 0.9789 - val_helper_output_loss: 0.8699 - val_output_mean_absolute_error: 0.7631 - val_helper_output_mean_absolute_error: 0.8699
Epoch 3/40
363/363 [==============================] - 6s 16ms/step - loss: 0.9851 - output_loss: 1.0113 - helper_output_loss: 0.8802 - output_mean_absolute_error: 0.7668 - helper_output_mean_absolute_error: 0.8802 - val_loss: 0.9543 - val

## 4_tensorboard

In [52]:
import datetime

# تابعی برای ایجاد نام فایل بر اساس تاریخ و زمان فعلی
def create_log_dir(base_dir, base_name):
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_dir = f"{base_dir}/{base_name}_{current_time}"
    return log_dir

# مسیر دلخواه پوشه و نام دلخواه فایل
base_dir = "./tb_log/callbacks"
base_name = "tb_log_test"

# ایجاد دایرکتوری لاگ
log_dir = create_log_dir(base_dir, base_name)

# استفاده از TensorBoard callback با مسیر لاگ منحصر به فرد
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)


In [53]:
history_conc=model_conc.fit(
    x=(X_train_s_1 , X_train_s_2 ),
    #چون ما دوتا خروجی داریم پس دوتا وای هم داریم 
    y=(y_train1,y_train1),
    epochs=40,
    validation_data=  ((X_validaion_s_1 , X_validaion_s_2) ,
                      (y_validaion,y_validaion)),
    callbacks=[model_Checkpoint_callback,
               earlystopping_callback_,
               MyCallback_mehr,
               tensorboard_callback
              ]
                )

Epoch 1/40
363/363 [==============================] - 6s 17ms/step - loss: 0.9759 - output_loss: 0.9999 - helper_output_loss: 0.8802 - output_mean_absolute_error: 0.7670 - helper_output_mean_absolute_error: 0.8802 - val_loss: 0.9520 - val_output_loss: 0.9725 - val_helper_output_loss: 0.8699 - val_output_mean_absolute_error: 0.7673 - val_helper_output_mean_absolute_error: 0.8699
Epoch 2/40
363/363 [==============================] - 6s 16ms/step - loss: 0.9738 - output_loss: 0.9972 - helper_output_loss: 0.8802 - output_mean_absolute_error: 0.7674 - helper_output_mean_absolute_error: 0.8802 - val_loss: 0.9526 - val_output_loss: 0.9733 - val_helper_output_loss: 0.8700 - val_output_mean_absolute_error: 0.7651 - val_helper_output_mean_absolute_error: 0.8700
Epoch 3/40
363/363 [==============================] - 6s 16ms/step - loss: 0.9770 - output_loss: 1.0012 - helper_output_loss: 0.8802 - output_mean_absolute_error: 0.7676 - helper_output_mean_absolute_error: 0.8802 - val_loss: 0.9541 - val

In [54]:
%load_ext tensorboard

In [55]:
%tensorboard --logdir Downloads/Video/tb_log/callbacks/tb_log_test_2024-12-24_22-05-02

## heperparameters optimization

In [ ]:
def ann_model(number_of_hidden_layers=1, number_of_neurons=10, learning_rate=0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=[8]))
    for _ in range(number_of_hidden_layers):
        model.add(keras.layers.Dense(number_of_neurons, activation='selu'))
    model.add(keras.layers.Dense(1))
    sgd = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=sgd)
    return model

In [ ]:
from scikeras.wrappers import KerasRegressor
# Wrap the model with KerasRegressor
keras_sikit_learn_regresion = KerasRegressor(model=ann_model)

از گرید سرچ رندم هم می توانید استفاده کنید در پایین مثل همانطور که در ماشین لرنینگ داشتیم و میتوانیم بازه تغریف کنیم که بین این اعداد به صورت تصادفی انتخاب کند
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

In [ ]:
# Define the parameter grid
param_grid = {
    'model__number_of_hidden_layers': [1, 3, 5],
    'model__number_of_neurons': [50, 100, 150],
    'model__learning_rate': [0.001, 0.01, 0.1],
}

In [ ]:
# Create the GridSearchCV object
keras_sikit_learn_regresion_GS = GridSearchCV(
    estimator=keras_sikit_learn_regresion,
    param_grid=param_grid,
    cv=3
)

In [ ]:
# Fit the GridSearchCV object
keras_sikit_learn_regresion_GS.fit(
    X_train_s,
    y_train1,
    validation_data=(X_validaion_s, y_validaion),
    callbacks=[keras.callbacks.EarlyStopping(patience=5)],
    epochs=2
)

Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 1.9032 - val_loss: 0.7194
Epoch 2/2
121/121 [==============================] - 0s 781us/step
Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 1.7570 - val_loss: 0.7549
Epoch 2/2
121/121 [==============================] - 0s 872us/step
Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 1.9115 - val_loss: 0.7429
Epoch 2/2
121/121 [==============================] - 0s 830us/step
Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 1.8760 - val_loss: 0.7774
Epoch 2/2
121/121 [==============================] - 0s 781us/step
Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 1.7798 - val_loss: 0.6852
Epoch 2/2
121/121 [==============================] - 0s 839us/step
Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 1.9468 - val_loss: 0.8787
Epoch 2/2
121/121 [==============================] - 0s 797us/step
Epoc

D:\anacoda\envs\a.i\lib\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "D:\anacoda\envs\a.i\lib\site-packages\scikeras\wrappers.py", line 1117, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "D:\anacoda\envs\a.i\lib\site-packages\scikeras\wrappers.py", line 1714, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\utils\_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "D:\anacoda\env

242/242 [==============================] - 1s 2ms/step - loss: 0.6543 - val_loss: 0.6017
Epoch 2/2
121/121 [==============================] - 0s 781us/step
Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 1.0937 - val_loss: 18.5310
Epoch 2/2
121/121 [==============================] - 0s 847us/step
Epoch 1/2


D:\anacoda\envs\a.i\lib\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "D:\anacoda\envs\a.i\lib\site-packages\scikeras\wrappers.py", line 1117, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "D:\anacoda\envs\a.i\lib\site-packages\scikeras\wrappers.py", line 1714, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\utils\_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "D:\anacoda\env

242/242 [==============================] - 1s 2ms/step - loss: 0.6969 - val_loss: 0.5100
Epoch 2/2
121/121 [==============================] - 0s 781us/step
Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 0.5802 - val_loss: 0.6006
Epoch 2/2
121/121 [==============================] - 0s 755us/step
Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 0.7051 - val_loss: 0.8404
Epoch 2/2
121/121 [==============================] - 0s 781us/step
Epoch 1/2
242/242 [==============================] - 1s 2ms/step - loss: 0.8365 - val_loss: 0.9989
Epoch 2/2
121/121 [==============================] - 0s 822us/step
Epoch 1/2


...

D:\anacoda\envs\a.i\lib\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "D:\anacoda\envs\a.i\lib\site-packages\scikeras\wrappers.py", line 1117, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "D:\anacoda\envs\a.i\lib\site-packages\scikeras\wrappers.py", line 1714, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\utils\_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "D:\anacoda\env

242/242 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 2/2
121/121 [==============================] - 0s 1ms/step
Epoch 1/2


D:\anacoda\envs\a.i\lib\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "D:\anacoda\envs\a.i\lib\site-packages\scikeras\wrappers.py", line 1117, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "D:\anacoda\envs\a.i\lib\site-packages\scikeras\wrappers.py", line 1714, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "D:\anacoda\envs\a.i\lib\site-packages\sklearn\utils\_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "D:\anacoda\env

363/363 [==============================] - 1s 2ms/step - loss: 0.5404 - val_loss: 0.4521
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.4162 - val_loss: 0.4028


GridSearchCV(cv=3,
             estimator=KerasRegressor(model=<function ann_model at 0x00000281575E3D30>),
             param_grid={'model__learning_rate': [0.001, 0.01, 0.1],
                         'model__number_of_hidden_layers': [1, 3, 5],
                         'model__number_of_neurons': [50, 100, 150]})

In [ ]:
keras_sikit_learn_regresion_GS.best_params_

{'model__learning_rate': 0.01,
 'model__number_of_hidden_layers': 5,
 'model__number_of_neurons': 50}

In [ ]:
keras_sikit_learn_regresion_GS.best_score_

0.6851803360367911

In [ ]:
model_gs = keras_sikit_learn_regresion_GS.best_estimator_.model

## hyper opt and hyperas and keras_tuner and Hyperband and skopt are another methode  to set hyper parameter 
### https://github.com/hyperopt/hyperopt 
### https://github.com/maxpumperla/hyperas
### https://github.com/keras-team/keras-tuner
### https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html
### https://github.com/scikit-optimize/scikit-optimize
### https://github.com/zygmuntz/hyperband